# Conexión a Bases de datos

**Autor:** Roberto Muñoz <br />
**E-mail:** <rmunoz@metricarts.com> <br />
**Github:** <https://github.com/rpmunoz> <br />


- Instalar el conector de MySQL para python

    `!pip install mysql-connector-python`
    
- Crear usuario en DB y dar permisos de lectura

 `GRANT SELECT ON chile.* TO 'user'@'%' IDENTIFIED BY 'metric2019';`
 
 `GRANT SELECT ON censo_2017.* TO 'user'@'%' IDENTIFIED BY 'metric2019';`
 
 `GRANT SELECT ON chile.* TO 'user'@'localhost' IDENTIFIED BY 'metric2019';`

In [2]:
!pip install mysql-connector-python

In [3]:
import numpy as np
import pandas as pd
import mysql.connector as sql

# Análisis de división geográfica en Chile

In [4]:
# Database args
db_user = 'user@tallerpython'
db_passwd = 'metric2019'
db_host = 'tallerpython.mysql.database.azure.com'
db_name = 'chile'

In [21]:
cnx = sql.connect(user=db_user, password=db_passwd, host=db_host, database=db_name)

Usando el cursor de SQL. Ojo que antes de cerrar el cursor tenemos que realizar las operaciones de lectura que necesitemos.

In [11]:
query= "SHOW TABLES;"

cursor = cnx.cursor(buffered=True)
cursor.execute(query)

for x in cursor:
    print(x)

cursor.close()

('comunas',)
('provincias',)
('regiones',)


True

Usando el método read_sql de Pandas

In [12]:
query= "SHOW TABLES;"
tablas=pd.read_sql(query, con=cnx)
tablas

,Tables_in_chile
0,comunas
1,provincias
2,regiones


Hagamos un select en la tabla regiones

In [13]:
query= "SELECT * from regiones;"
regiones=pd.read_sql(query, con=cnx)
regiones

,region_id,region_nombre,region_ordinal
0,1,Arica y Parinacota,XV
1,2,Tarapacá,I
2,3,Antofagasta,II
3,4,Atacama,III
4,5,Coquimbo,IV
5,6,Valparaiso,V
6,7,Metropolitana de Santiago,RM
7,8,Libertador General Bernardo O'Higgins,VI
8,9,Maule,VII
9,10,Biobío,VIII


La siguiente consulta está parametrizada. Fíjese cómo se declaran los parámetros dentro de la cadena de la consulta, y cómo después se ingresan, en el comando read_sql de Pandas.

In [14]:
query= "SELECT * from regiones WHERE region_id BETWEEN %s and %s;"
regiones=pd.read_sql(query, params=(2,8), con=cnx)
regiones.head()

,region_id,region_nombre,region_ordinal
0,2,Tarapacá,I
1,3,Antofagasta,II
2,4,Atacama,III
3,5,Coquimbo,IV
4,6,Valparaiso,V


Como verá en la siguiente consulta, estamos utilizando cosas aprendidas en los tutoriales anteriores. Estamos armando dinámicamente la cadena que respresenta una consulta. Específicamente, utilizamos una lista con id's de regiones que nos interesan, lo convertimos a un objeto `array` de numpy (np.array), después aplicamos la función `map` para aplicar la función `str` al array, y finalmente, concatenamos los resultados con el caracter ",". **Todo en una línea**.
¿En cuantas líneas podría hacer esto en Java?

In [17]:
','.join(["Hola","Mundo","Como","Estan"])

'Hola,Mundo,Como,Estan'

In [19]:
region_ids=['5',8,10]
query_region= ', '.join(map(str, np.array(region_ids)))
query_region

'5, 8, 10'

In [22]:
region_ids=[5,8,10]

query= "SELECT * from regiones WHERE region_id IN ("

query_region= ', '.join(map(str, np.array(region_ids)))

query= query + query_region + ");"

print(query)

regiones=pd.read_sql(query, con=cnx)
regiones

SELECT * from regiones WHERE region_id IN (5, 8, 10);


,region_id,region_nombre,region_ordinal
0,5,Coquimbo,IV
1,8,Libertador General Bernardo O'Higgins,VI
2,10,Biobío,VIII


## Uso de pandas.merge()

In [23]:
query= "SELECT * from regiones;"
regiones=pd.read_sql(query, con=cnx)
regiones

,region_id,region_nombre,region_ordinal
0,1,Arica y Parinacota,XV
1,2,Tarapacá,I
2,3,Antofagasta,II
3,4,Atacama,III
4,5,Coquimbo,IV
5,6,Valparaiso,V
6,7,Metropolitana de Santiago,RM
7,8,Libertador General Bernardo O'Higgins,VI
8,9,Maule,VII
9,10,Biobío,VIII


In [24]:
query= "SELECT * from provincias;"
provincias=pd.read_sql(query, con=cnx)
provincias.head()

,provincia_id,provincia_nombre,region_id
0,1,Arica,1
1,2,Parinacota,1
2,3,Iquique,2
3,4,El Tamarugal,2
4,5,Antofagasta,3


In [25]:
query= "SELECT * from comunas;"
comunas=pd.read_sql(query, con=cnx)
comunas.head()

,comuna_id,comuna_nombre,provincia_id
0,1,Arica,1
1,2,Camarones,1
2,3,General Lagos,2
3,4,Putre,2
4,5,Alto Hospicio,3


In [26]:
regiones_provincias = pd.merge(regiones, provincias, how='outer')
regiones_provincias.head()

,region_id,region_nombre,region_ordinal,provincia_id,provincia_nombre
0,1,Arica y Parinacota,XV,1,Arica
1,1,Arica y Parinacota,XV,2,Parinacota
2,2,Tarapacá,I,3,Iquique
3,2,Tarapacá,I,4,El Tamarugal
4,3,Antofagasta,II,5,Antofagasta


Cerramos la conexión al servidor de MySQL.

In [27]:
cnx.close()

# Análisis los datos del Censo 2017

In [28]:
# Database args
db_user = 'user@tallerpython'
db_passwd = 'metric2019'
db_host = 'tallerpython.mysql.database.azure.com'
db_name = 'censo_2017'

In [29]:
cnx = sql.connect(user=db_user, password=db_passwd, host=db_host, database=db_name)

In [30]:
query= "SHOW TABLES;"
tablas=pd.read_sql(query, con=cnx)
tablas

,Tables_in_censo_2017
0,comunas


In [31]:
query= "DESCRIBE comunas;"
censo_describe=pd.read_sql(query, con=cnx)
censo_describe

,Field,Type,Null,Key,Default,Extra
0,orden,text,YES,,None,
1,nombre region,text,YES,,None,
2,codigo region,text,YES,,None,
3,nombre provincia,text,YES,,None,
4,codigo provincia,text,YES,,None,
5,nombre comuna,text,YES,,None,
6,codigo comuna,text,YES,,None,
7,edad,text,YES,,None,
8,total poblacion efectivamente censada,int(10) unsigned,YES,,None,
9,hombres,int(10) unsigned,YES,,None,


In [32]:
query= "SELECT count(*) from comunas;"
censo_count=pd.read_sql(query, con=cnx)
censo_count.head()

,count(*)
0,34946


In [33]:
query= "SELECT * from comunas;"
censo_df = pd.read_sql(query, con=cnx)
censo_df.head()

,orden,nombre region,codigo region,nombre provincia,codigo provincia,nombre comuna,codigo comuna,edad,total poblacion efectivamente censada,hombres,mujeres,total area urbana,hombres area urbana,mujeres area urbana,total area rural,hombres area rural,mujeres area rural
0,103,arica y parinacota,15,arica,151,arica,15101,0,2802,1416,1386,2557,1292,1265,245,124,121
1,104,arica y parinacota,15,arica,151,arica,15101,1,3084,1562,1522,2841,1428,1413,243,134,109
2,105,arica y parinacota,15,arica,151,arica,15101,2,3339,1743,1596,3086,1600,1486,253,143,110
3,106,arica y parinacota,15,arica,151,arica,15101,3,3347,1720,1627,3104,1596,1508,243,124,119
4,107,arica y parinacota,15,arica,151,arica,15101,4,3321,1682,1639,3080,1566,1514,241,116,125


In [35]:
#censo_df

### Sumamos la poblacion en Chile por sexo

In [36]:
censo_df['hombres'].sum()

8601989

In [37]:
censo_df['mujeres'].sum()

8972014

### Extraemos la poblacion de la region metropolitana

In [38]:
censo_df['nombre region'].unique()

array(['arica y parinacota', 'tarapacá', 'antofagasta', 'atacama',
       'coquimbo', 'valparaíso', 'metropolitana de santiago',
       "libertador general bernardo o'higgins", 'maule', 'ñuble',
       'biobío', 'la araucanía', 'los ríos', 'los lagos',
       'aysén del general carlos ibáñez del campo',
       'magallanes y de la antártica chilena'], dtype=object)

In [43]:
names=['codigo region','edad']
censo_rm_df = censo_df[ (censo_df['nombre region'] == 'metropolitana de santiago') | (censo_df['nombre region'] == 'arica y parinacota')  ][names]
censo_rm_df.head()

,codigo region,edad
0,15,0
1,15,1
2,15,2
3,15,3
4,15,4


In [42]:
filtro = censo_rm_df[['codigo region','edad']]
filtro.head()<

,codigo region,edad
0,15,0
1,15,1
2,15,2
3,15,3
4,15,4


In [40]:
len(censo_rm_df)

5656

Cerramos la conexión al servidor de MySQL. **¡¡No olvide hacer esto!!**

In [ ]:
cnx.close()

## Ejercicios

### 1. ¿Cuáles son las 5 comunas de la Región metropolitana con mayor número de habitantes?

### 2. ¿Cuál es la región de Chile con el mayor porcentaje de población rural?



In [45]:
censo_rm_df = censo_df[ censo_df['nombre region'] == 'metropolitana de santiago'] 
censo_rm_df.sort_values('total poblacion efectivamente censada', ascending=False).head()

,orden,nombre region,codigo region,nombre provincia,codigo provincia,nombre comuna,codigo comuna,edad,total poblacion efectivamente censada,hombres,mujeres,total area urbana,hombres area urbana,mujeres area urbana,total area rural,hombres area rural,mujeres area rural
8309,8494,metropolitana de santiago,13,santiago,131,santiago,13101,27,14116,7498,6618,14116,7498,6618,0,0,0
8310,8495,metropolitana de santiago,13,santiago,131,santiago,13101,28,14101,7584,6517,14101,7584,6517,0,0,0
8311,8496,metropolitana de santiago,13,santiago,131,santiago,13101,29,13961,7480,6481,13961,7480,6481,0,0,0
8312,8497,metropolitana de santiago,13,santiago,131,santiago,13101,30,13406,7334,6072,13406,7334,6072,0,0,0
8308,8493,metropolitana de santiago,13,santiago,131,santiago,13101,26,13039,6727,6312,13039,6727,6312,0,0,0


In [57]:
censo_rm_df.groupby('nombre comuna').\
agg({'total poblacion efectivamente censada':'sum','hombres':'sum', 'mujeres':'sum', 'total area urbana':'mean'}).\
sort_values('total poblacion efectivamente censada', ascending=False).head()


,total poblacion efectivamente censada,hombres,mujeres,total area urbana
nombre comuna,,,,
puente alto,568106,275147,292959,5624.693069
maipú,521627,250792,270835,5130.633663
santiago,404495,206678,197817,4004.900990
la florida,366916,175693,191223,3631.673267
san bernardo,301313,147800,153513,2933.148515
